In [2]:
import time
# from langchain_ollama.llms import OllamaLLM
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from function_call_tool import tools
load_dotenv()

True

In [ ]:
"""
통일된 interface 통해 function calling test

ollama 모델 사용:
1. download ollama
$ curl -fsSL https://ollama.com/install.sh | sh
2. pull model with model name in https://ollama.com/search
$ ollama run {model_name}
"""

MODEL_LIST = [
    # "gpt-4o-mini",
    # "gpt-4o",
    # ollama interface
    "qwen2.5:7b",
    # "qwen2.5:14b",
    # "llama3.1:8b",
    # huggingface
    # "Jungwonchang/Ko-QWEN-7B-Chat-LoRA",
    # "etri-xainlp/qwen-ko-14b-instruct",
    # "Saxo/Linkbricks-Horizon-AI-Korean-llama3.1-sft-rlhf-dpo-8B",
    # "SEOKDONG/llama3.1_korean_v1.1_sft_by_aidx"
]

QUESTION_LIST = [
    "주문 조회해줘",
    "내 등급 조회",
    "스트라이프 셔츠 주문한거 언제와",
    "내 등급이랑 주문정보 알려줘", # multi tool use
    "한국의 수도가 어디야"
]

# llm_invoke_callbacks = [ConsoleCallbackHandler()]
llm_invoke_callbacks = []

tool_dict = {t.name:t for t in tools}

def get_model(model_name):
    if "gpt" in model_name:
        model = ChatOpenAI(model_name=model_name)
    elif ":" in model_name:
        model = ChatOllama(model=model_name, base_url = "http://192.168.1.203:11434")
    else:
        model = HuggingFacePipeline.from_model_id(
            model_id=model_name,
            task="text-generation",
            pipeline_kwargs={"max_new_tokens":512}
        )
    llm_with_tools = model.bind_tools(tools)
    return llm_with_tools

def invoke_model(query, llm):
    messages = [HumanMessage(query)]

    ai_msg = llm.invoke(messages, config={'callbacks': llm_invoke_callbacks})
    messages.append(ai_msg)
    # return response
    if ai_msg.content=='':
        # tool call
        for tool_call in ai_msg.tool_calls:
            selected_tool = tool_dict[tool_call['name']]
            print(f"* {selected_tool.name}() tool 사용중...")
            tool_msg = selected_tool.invoke(tool_call)
            messages.append(tool_msg)
        response2 = llm.invoke(messages, config={'callbacks': llm_invoke_callbacks})
        answer = response2.content
    else:
        answer = ai_msg.content
    return answer


def main(model_name):
    print(f"\n\n< MODEL : {model_name} >\n")
    llm = get_model(model_name)

    for q in QUESTION_LIST:
        print(f"Q: {q}")
        t0 = time.time()
        answer = invoke_model(q, llm)
        print(f"A : {answer}\n")
        took = time.time() - t0
        print(f"\nTook {round(took, 4)}s")
        print("-"*50) 



for model in MODEL_LIST:
    main(model)

